# Create pf3k release5

This notebook was run from malsrv2

This notebook requires the output from notebook 20160120_vqsr_delta_exploration.ipynb (which was run on Richard'd MacBook, and results copied over to malsrv2)

In [2]:
%run imports.ipynb
%run _shared_setup.ipynb

In [3]:
# !apt-get update
# !apt-get install tabix

# Setup

In [4]:
release5_intermediate_files_dir = '/nfs/team112_internal/rp7/data/pf3k/release5/intermediate_files'
release5_final_files_dir = '/nfs/team112_internal/production/release_build/Pf3K/pilot_5_0'


In [5]:
if not os.path.exists(release5_intermediate_files_dir):
    os.makedirs(release5_intermediate_files_dir)
if not os.path.exists(os.path.dirname(CHROM_VCF_FNS['release5']['Pf3D7_01_v3'])):
    os.makedirs(os.path.dirname(CHROM_VCF_FNS['release5']['Pf3D7_01_v3']))

In [6]:
CHROM_VCF_FNS['release5']['Pf3D7_01_v3']

'/nfs/team112_internal/production/release_build/Pf3K/pilot_5_0/SNP_INDEL_Pf3D7_01_v3.combined.filtered.vcf.gz'

# Plan
- Make SNP and indel files for each chrom using SelectVariants
- Add VQSLOD scores to these using ApplyRecalibration with outputs of 20160120_vqsr_delta_exploration.ipynb
- Combine SNP and indel files using CombineVariants
- Create final filtered versions using VariantFiltration
- Merge all chroms using bcftools concat

In [20]:
GenomeAnalysisTK="/software/jre1.7.0_25/bin/java -jar /nfs/team112_internal/production/tools/bin/gatk/GenomeAnalysisTK-3.5/GenomeAnalysisTK.jar"
bcftools="/nfs/team112_internal/production/tools/bin/bcftools_git"

In [21]:
!{GenomeAnalysisTK}

##### ERROR ------------------------------------------------------------------------------------------
##### ERROR A USER ERROR has occurred (version 3.5-0-g36282e4): 
##### ERROR
##### ERROR This means that one or more arguments or inputs in your command are incorrect.
##### ERROR The error message below tells you what is the problem.
##### ERROR
##### ERROR If the problem is an invalid argument, please check the online documentation guide
##### ERROR (or rerun your command with --help) to view allowable command-line arguments for this tool.
##### ERROR
##### ERROR Visit our website and forum for extensive documentation and answers to 
##### ERROR commonly asked questions http://www.broadinstitute.org/gatk
##### ERROR
##### ERROR Please do NOT post this error to the GATK forum unless you have really tried to fix it yourself.
##### ERROR
##### ERROR MESSAGE: Argument with name '--analysis_type' (-T) is missing.
##### ERROR ---------------------------------------------------------------

In [22]:
!{bcftools}


Program: bcftools (Tools for variant calling and manipulating VCFs and BCFs)
Version: 1.1-113-gd991f3f (using htslib 1.1-104-g948a68c)

Usage:   bcftools <command> <argument>

Commands:

 -- Indexing
    index        index VCF/BCF files

 -- VCF/BCF manipulation
    annotate     annotate and edit VCF/BCF files
    concat       concatenate VCF/BCF files from the same set of samples
    convert      convert VCF/BCF files to different formats and back
    isec         intersections of VCF/BCF files
    merge        merge VCF/BCF files files from non-overlapping sample sets
    norm         left-align and normalize indels
    plugin       user-defined plugins
    query        transform VCF/BCF into user-defined formats
    reheader     modify VCF/BCF header, change sample names
    view         VCF/BCF conversion, view, subset and filter VCF/BCF files

 -- VCF/BCF analysis
    call         SNP/indel calling
    consensus    create consensus sequence by applying VCF variants
    filter    

In [ ]:
# Make SNP and indel files for each chrom using SelectVariants
chroms = CHROM_VCF_FNS['release5'].keys()

for chrom in chroms:
    input_vcf_fn = CHROM_VCF_FNS['interim5'][chrom]
    snps_fn = "%s/snps_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    indels_fn = "%s/indels_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    if not os.path.exists("%s.tbi" % input_vcf_fn):
        !bcftools index --tbi {input_vcf_fn}
    !{GenomeAnalysisTK} \
        -T SelectVariants \
        -R {GENOME_FN} \
        -V {input_vcf_fn} \
        -selectType SNP \
        -o /dev/stdout | bgzip > {snps_fn}
    !{bcftools} index --tbi {snps_fn}
    !{GenomeAnalysisTK} \
        -T SelectVariants \
        -R {GENOME_FN} \
        -V {input_vcf_fn} \
        -xlSelectType SNP \
        -o /dev/stdout | bgzip > {indels_fn}
    !{bcftools} index --tbi {indels_fn}


INFO  12:12:01,662 HelpFormatter - -------------------------------------------------------------------------------- 
INFO  12:12:01,668 HelpFormatter - The Genome Analysis Toolkit (GATK) v3.5-0-g36282e4, Compiled 2015/11/25 04:03:56 
INFO  12:12:01,669 HelpFormatter - Copyright (c) 2010 The Broad Institute 
INFO  12:12:01,669 HelpFormatter - For support and documentation go to http://www.broadinstitute.org/gatk 
INFO  12:12:01,674 HelpFormatter - Program Args: -T SelectVariants -R /lustre/scratch109/malaria/pf3k_methods/resources/Pfalciparum.genome.fasta -V /lustre/scratch109/malaria/pf3k_methods/output/1/f/5/c/394734/1_gatk_variant_filter_gatk3/SNP_INDEL_Pf3D7_01_v3.combined.filtered.vcf.gz -selectType SNP -o /dev/stdout 
INFO  12:12:01,682 HelpFormatter - Executing as rp7@malsrv2 on Linux 3.2.0-75-generic amd64; Java HotSpot(TM) 64-Bit Server VM 1.7.0_25-b15. 
INFO  12:12:01,683 HelpFormatter - Date/Time: 2016/02/04 12:12:01 
INFO  12:12:01,683 HelpFormatter - -----------------------

In [16]:
tranches_snp_fn = '/nfs/team112_internal/rp7/data/pf3k/interim5/vqsr/no_ranksum_3_5_mqcap70/SNP.tranches'
recal_snp_fn = '/nfs/team112_internal/rp7/data/pf3k/interim5/vqsr/no_ranksum_3_5_mqcap70/SNP.recal'
tranches_indel_fn = '/nfs/team112_internal/rp7/data/pf3k/interim5/vqsr/gatk_rec_indel/INDEL.tranches'
recal_indel_fn = '/nfs/team112_internal/rp7/data/pf3k/interim5/vqsr/gatk_rec_indel/INDEL.recal'

for chrom in chroms:
    snps_fn = "%s/snps_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    indels_fn = "%s/indels_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    snps_recalibrated_fn = "%s/snps_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    indels_recalibrated_fn = "%s/indels_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    
    if not os.path.exists(snps_recalibrated_fn):
        !{GenomeAnalysisTK} \
            -T ApplyRecalibration \
            -R {GENOME_FN} \
            -input {snps_fn} \
            -tranchesFile {tranches_snp_fn} \
            -recalFile {recal_snp_fn} \
            --ts_filter_level 99.5 \
            -mode SNP \
            --ignore_all_filters \
            -o /dev/stdout | bgzip > {snps_recalibrated_fn}
    if not os.path.exists("%s.tbi" % snps_recalibrated_fn):
        !{bcftools} index --tbi {snps_recalibrated_fn}
        
    if not os.path.exists(indels_recalibrated_fn):
        !{GenomeAnalysisTK} \
            -T ApplyRecalibration \
            -R {GENOME_FN} \
            -input {indels_fn} \
            -tranchesFile {tranches_indel_fn} \
            -recalFile {recal_indel_fn} \
            --ts_filter_level 99.0 \
            -mode INDEL \
            --ignore_all_filters \
            -o /dev/stdout | bgzip > {indels_recalibrated_fn}
    if not os.path.exists("%s.tbi" % indels_recalibrated_fn):
        !{bcftools} index --tbi {indels_recalibrated_fn}


INFO  20:38:21,670 HelpFormatter - -------------------------------------------------------------------------------- 
INFO  20:38:21,672 HelpFormatter - The Genome Analysis Toolkit (GATK) v3.5-0-ge91472d, Compiled 2016/01/14 14:23:38 
INFO  20:38:21,677 HelpFormatter - Copyright (c) 2010 The Broad Institute 
INFO  20:38:21,677 HelpFormatter - For support and documentation go to http://www.broadinstitute.org/gatk 
INFO  20:38:21,680 HelpFormatter - Program Args: -T ApplyRecalibration -R /lustre/scratch109/malaria/pf3k_methods/resources/Pfalciparum.genome.fasta -input /nfs/team112_internal/rp7/data/pf3k/release5/intermediate_files/snps_Pf3D7_01_v3.vcf.gz -tranchesFile /nfs/team112_internal/rp7/data/pf3k/interim5/vqsr/no_ranksum_3_5_mqcap70/SNP.tranches -recalFile /nfs/team112_internal/rp7/data/pf3k/interim5/vqsr/no_ranksum_3_5_mqcap70/SNP.recal --ts_filter_level 99.5 -mode SNP -o /dev/stdout 
INFO  20:38:21,691 HelpFormatter - Executing as root@fed69f1cebd6 on Linux 4.1.13-boot2docker amd

In [17]:
for chrom in chroms:
    snps_recalibrated_fn = "%s/snps_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    indels_recalibrated_fn = "%s/indels_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    combined_recalibrated_fn = "%s/snps_indels_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    
    if not os.path.exists(combined_recalibrated_fn):
        !GenomeAnalysisTK \
            -T CombineVariants \
            -R {GENOME_FN} \
            --variant:snp {snps_recalibrated_fn} \
            --variant:indel {indels_recalibrated_fn} \
            -genotypeMergeOptions PRIORITIZE \
            -priority snp,indel \
            -o /dev/stdout | bgzip > {combined_recalibrated_fn}
            
    if not os.path.exists("%s.tbi" % combined_recalibrated_fn):
        !bcftools index --tbi {combined_recalibrated_fn}


INFO  03:12:35,039 HelpFormatter - -------------------------------------------------------------------------------- 
INFO  03:12:35,041 HelpFormatter - The Genome Analysis Toolkit (GATK) v3.5-0-ge91472d, Compiled 2016/01/14 14:23:38 
INFO  03:12:35,044 HelpFormatter - Copyright (c) 2010 The Broad Institute 
INFO  03:12:35,044 HelpFormatter - For support and documentation go to http://www.broadinstitute.org/gatk 
INFO  03:12:35,047 HelpFormatter - Program Args: -T CombineVariants -R /lustre/scratch109/malaria/pf3k_methods/resources/Pfalciparum.genome.fasta --variant:snp /nfs/team112_internal/rp7/data/pf3k/release5/intermediate_files/snps_recalibrated_Pf3D7_01_v3.vcf.gz --variant:indel /nfs/team112_internal/rp7/data/pf3k/release5/intermediate_files/indels_recalibrated_Pf3D7_01_v3.vcf.gz -genotypeMergeOptions PRIORITIZE -priority snp,indel -o /dev/stdout 
INFO  03:12:35,060 HelpFormatter - Executing as root@fed69f1cebd6 on Linux 4.1.13-boot2docker amd64; OpenJDK 64-Bit Server VM 1.7.0_91-

In [36]:
for chrom in chroms:
    combined_recalibrated_fn = "%s/snps_indels_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    combined_filtered_fn = "%s/SNP_INDEL_%s.combined.filtered.vcf.gz" % (release5_final_files_dir, chrom)

    if not os.path.exists(combined_filtered_fn):
#         gatk = sh.Command('GenomeAnalysisTK')
#         bgzip = sh.Command('bgzip')
#         bgzip(gatk(
#             "-T", "VariantFiltration",
#             "-R", GENOME_FN,
#             "--variant", combined_recalibrated_fn,
#             "--filterName", "\"Low_VQSLOD\"", "--filterExpression", "\"VQSLOD <= 0.0\"",
#             "--filterName", "\"Centromere\"", "--filterExpression", "\"RegionType == 'Centromere'\"",
#             "--filterName", "\"InternalHypervariable\"", "--filterExpression", "\"RegionType == 'InternalHypervariable'\"",
#             "--filterName", "\"SubtelomericHypervariable\"", "--filterExpression", "\"RegionType == 'SubtelomericHypervariable'\"",
#             "--filterName", "\"SubtelomericRepeat\"", "--filterExpression", "\"RegionType == 'SubtelomericRepeat'\"",
#             "--invalidatePreviousFilters",
#             "-o", "/dev/stdout"
#         ), _out=combined_filtered_fn)
        !GenomeAnalysisTK \
            -T VariantFiltration \
            -R {GENOME_FN} \
            --variant {combined_recalibrated_fn} \
            --filterName "Low_VQSLOD" --filterExpression "VQSLOD<=0.0" \
            --filterName "Centromere" --filterExpression "RegionType=='Centromere'" \
            --filterName "InternalHypervariable" --filterExpression "RegionType=='InternalHypervariable'" \
            --filterName "SubtelomericHypervariable" --filterExpression "RegionType=='SubtelomericHypervariable'" \
            --filterName "SubtelomericRepeat" --filterExpression "RegionType=='SubtelomericRepeat'" \
            --invalidatePreviousFilters \
            -o /dev/stdout | bgzip > {combined_filtered_fn}

    if not os.path.exists("%s.tbi" % combined_filtered_fn):
        !bcftools index --tbi {combined_filtered_fn}


INFO  22:04:39,367 HelpFormatter - -------------------------------------------------------------------------------- 
INFO  22:04:39,369 HelpFormatter - The Genome Analysis Toolkit (GATK) v3.5-0-ge91472d, Compiled 2016/01/14 14:23:38 
INFO  22:04:39,372 HelpFormatter - Copyright (c) 2010 The Broad Institute 
INFO  22:04:39,372 HelpFormatter - For support and documentation go to http://www.broadinstitute.org/gatk 
INFO  22:04:39,375 HelpFormatter - Program Args: -T VariantFiltration -R /lustre/scratch109/malaria/pf3k_methods/resources/Pfalciparum.genome.fasta --variant /nfs/team112_internal/rp7/data/pf3k/release5/intermediate_files/snps_indels_recalibrated_Pf3D7_01_v3.vcf.gz --filterName Low_VQSLOD --filterExpression VQSLOD<=0.0 --filterName Centromere --filterExpression RegionType=='Centromere' --filterName InternalHypervariable --filterExpression RegionType=='InternalHypervariable' --filterName SubtelomericHypervariable --filterExpression RegionType=='SubtelomericHypervariable' --filte

In [37]:
input_files = ' '.join(
    ["%s/SNP_INDEL_%s.combined.filtered.vcf.gz" % (release5_final_files_dir, chrom) for chrom in CHROM_VCF_FNS['interim5'].keys()]
)
output_vcf_fn = "%s/SNP_INDEL_WG.combined.filtered.vcf.gz" % (release5_final_files_dir)
if not os.path.exists(output_vcf_fn):
    !bcftools concat --output-type z --output {output_vcf_fn} {input_files}
    !bcftools index --tbi {output_vcf_fn}
